# Задание 1

Генерация данных о студентах и их успеваемости

    Генерация данных:
        Сгенерируйте случайные данные о студентах, включая их идентификаторы (ID), имена, возрасты и города.
        Создайте данные о предметах (курсах), включая их идентификаторы (ID), названия и описания.
        Сгенерируйте данные об оценках студентов по предметам, включая их ID студента, ID предмета и оценку (например, от 1 до 10).

    Создание DataFrame:
        Создайте DataFrame для каждого набора данных (студенты, предметы, оценки) с использованием библиотеки PySpark.

    Обработка данных:
        Посчитайте средний возраст студентов.
        Определите количество студентов из каждого города.
        Найдите средний балл студентов по всем предметам.
        Определите самый популярный предмет (предмет, который выбрали наибольшее количество студентов).


### Импортируем библиотеки

In [18]:
import random
from faker import Faker
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count
from pyspark.sql.types import IntegerType, StringType, FloatType

### Инициализация SparkSession, Faker

In [19]:
spark = SparkSession.builder \
    .appName("Student Data Generation") \
    .getOrCreate()

fake = Faker('ru_RU')

### Количество студентов, курсов и оценок

In [20]:
num_students = int(input("Enter the number of students to generate: "))
num_courses = int(input("Enter the number of courses to generate: "))
num_grades = int(input("Enter the number of grades to generate: "))

# Генерация данных о студентах
students = [(i, fake.name(), random.randint(18, 25), fake.city()) for i in range(1, num_students + 1)]
df_students = spark.createDataFrame(students, ['StudentID', 'Name', 'Age', 'City'])

# Генерация данных о предметах
courses = [(i, fake.word().capitalize(), fake.sentence(nb_words=6)) for i in range(1, num_courses + 1)]
df_courses = spark.createDataFrame(courses, ['CourseID', 'CourseName', 'Description'])

# Генерация данных об оценках
grades = [(random.randint(1, num_students), random.randint(1, num_courses), random.randint(1, 10)) for _ in range(num_grades)]
df_grades = spark.createDataFrame(grades, ['StudentID', 'CourseID', 'Grade'])

## Обработка данных

### 1. Средний возраст студентов

In [21]:
avg_age = df_students.select(avg('Age')).first()[0]
print(f"Средний возраст студентов: {avg_age:.2f}")

Средний возраст студентов: 21.51


### 2. Количество студентов из каждого города

In [22]:
students_per_city = df_students.groupBy('City').count()
print("Количество студентов из каждого города:")
students_per_city.show()

Количество студентов из каждого города:
+--------------------+-----+
|                City|count|
+--------------------+-----+
|           д. Шумиха|    3|
|          клх Яхрома|    1|
|    г. Камень-на-Оби|    1|
|        д. Приозерск|    2|
|        п. Кондопога|    1|
|   г. Красная Поляна|    1|
|ст. Набережные Челны|    1|
|       клх Миллерово|    1|
|         д. Челюскин|    1|
|         д. Адыгейск|    2|
|            к. Тулун|    1|
|       к. Домодедово|    1|
|    к. Солнечногорск|    1|
|  г. Верхнее Пенжино|    1|
|          п. Джейрах|    1|
|       п. Мыс Шмидта|    2|
|д. Переславль-Зал...|    1|
|           п. Палана|    1|
|      д. Новосибирск|    1|
|         г. Качканар|    3|
+--------------------+-----+
only showing top 20 rows


### 3. Средний балл студентов по всем предметам

In [23]:
avg_grade = df_grades.select(avg('Grade')).first()[0]
print(f"Средний балл студентов по всем предметам: {avg_grade:.2f}")

Средний балл студентов по всем предметам: 6.88


### 4. Самый популярный предмет

In [24]:
popular_course = df_grades.groupBy('CourseID').count().orderBy('count', ascending=False).first()
course_name = df_courses.filter(df_courses.CourseID == popular_course['CourseID']).select('CourseName').first()[0]
print(f"Самый популярный предмет: {course_name} с количеством студентов: {popular_course['count']}")

Самый популярный предмет: Настать с количеством студентов: 2


### Сохранение данных в CSV

In [26]:
df_students.coalesce(1).write.csv('students.csv', header=True)
df_courses.coalesce(1).write.csv('courses.csv', header=True)
df_grades.coalesce(1).write.csv('grades.csv', header=True)

### Остановка SparkSession


In [27]:
spark.stop()